# Task 1: Response to Data analysis and feature engineering
使用pandas导入训练数据集，并分析特征和标签的分布情况。

In [ ]:
import platform
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA, FastICA, TruncatedSVD
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.feature_selection import SelectKBest, f_classif

# 设置随机种子和中文显示
np.random.seed(42)
system = platform.system()
if system == 'Darwin':
    plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
elif system == 'Windows':
    plt.rcParams['font.sans-serif'] = ['SimHei']
elif system == 'Linux':
    plt.rcParams['font.sans-serif'] = ['Noto Sans CJK SC']
else:
    plt.rcParams['font.sans-serif'] = ['SimHei']

df = pd.read_csv('./student_data.csv')

grade_models = {
    'pca_model': {},
    'bayes_models': {},
    'random_forest_models': {}
}

mapping = {1: 'A', 2: 'B', 3: 'C', 4: 'D'}
if df['Programme'].dtype == 'int64' or df['Programme'].iloc[0] in [1, 2, 3, 4]:
    df['Programme'] = df['Programme'].map(mapping)

def process_data(df, mode='train', preprocessors=None):
    if 'Index' in df.columns:
        df = df.drop('Index', axis=1)
    numeric_df = df.select_dtypes(include=['float64', 'int64'])
    feature_sets = {}

    if mode == 'train':
        preprocessors = {'scalers': {}, 'columns': {}}

    # 1. 考试分数
    if mode == 'train':
        exam_cols = [col for col in numeric_df.columns if 'Q' in col]
        if not exam_cols:
            exam_cols = numeric_df.columns[-5:].tolist()
        preprocessors['columns']['考试分数'] = exam_cols
    else:
        exam_cols = preprocessors['columns']['考试分数']
    # 补齐缺失列并保证顺序
    for col in exam_cols:
        if col not in numeric_df.columns:
            numeric_df[col] = 0
    feature_sets['考试分数'] = numeric_df[exam_cols].values

    # 2. 基本信息
    basic_patterns = ['性别', 'Gender', 'sex', 'Total', '总分', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5']
    if mode == 'train':
        basic_cols = []
        for pat in basic_patterns:
            basic_cols += [col for col in numeric_df.columns if pat.lower() in col.lower()]
        basic_cols = list(dict.fromkeys(basic_cols))
        if not basic_cols:
            basic_cols = numeric_df.columns[:2].tolist()
        preprocessors['columns']['去除年级'] = basic_cols
    else:
        basic_cols = preprocessors['columns']['去除年级']
    for col in basic_cols:
        if col not in numeric_df.columns:
            numeric_df[col] = 0
    feature_sets['去除年级'] = numeric_df[basic_cols].values

    # 3. 全部特征（排除programme列）
    if mode == 'train':
        programme_cols = [col for col in numeric_df.columns if 'programme' in col.lower() or 'program' in col.lower()]
        all_cols = [col for col in numeric_df.columns if col not in programme_cols]
        preprocessors['columns']['全部特征'] = all_cols
    else:
        all_cols = preprocessors['columns']['全部特征']
    for col in all_cols:
        if col not in numeric_df.columns:
            numeric_df[col] = 0
    feature_sets['全部特征'] = numeric_df[all_cols].values

    # 标准化
    for name, data in feature_sets.items():
        if mode == 'train':
            scaler = StandardScaler()
            feature_sets[name] = scaler.fit_transform(data)
            preprocessors['scalers'][name] = scaler
        else:
            feature_sets[name] = preprocessors['scalers'][name].transform(data)

    if mode == 'train':
        return feature_sets, preprocessors
    else:
        return feature_sets

导入数据后，查看数据集的基本信息。处理数据缺失
## 特征转换
应用三种不同的数据转换方法：标准化缩放、PCA降维和独立成分分析(ICA)

In [ ]:

# 去除索引列（如果存在）
if 'Index' in df.columns:
    df = df.drop('Index', axis=1)

print(f"数据集形状: {df.shape}")
print("\n数据集前5行:")
display(df.head())
print("\n数据集信息:")
display(df.info())
print(f"\n缺失值情况:")
display(df.isnull().sum())

# 只保留数值特征
numeric_df = df.select_dtypes(include=['float64', 'int64'])
feature_sets, preprocessors = process_data(df, mode='train')

print("创建的特征集:")
for name, features in feature_sets.items():
    print(f"- {name}: 形状 {features.shape}")

# 检查特征集并标准化
for name, data in feature_sets.items():
    print(f"\n特征集: {name}")
    print(f"数据集形状: {data.shape}")
    print("前5行:")
    print(pd.DataFrame(data[:5]))
    # 标准化
    scaler = StandardScaler()
    feature_sets[name] = scaler.fit_transform(data)
print("\n数据集统计描述:")
display(df.describe())

## 特征转换
应用三种不同的数据转换方法：标准化缩放、PCA降维和独立成分分析(ICA)

In [ ]:


from sklearn.manifold import TSNE

feature_set = feature_sets
feature_sets = {}
for name, X in feature_set.items():
   # 转换1: 归一化 (MinMaxScaler)
    min_max_scaler = MinMaxScaler()
    X_minmax = min_max_scaler.fit_transform(X)
    feature_sets['归一化_' + name] = X_minmax
    print(f"{name} 已完成归一化转换")

    # 转换2: 标准化 (StandardScaler)
    std_scaler = StandardScaler()
    X_std = std_scaler.fit_transform(X)
    feature_sets['标准化_' + name] = X_std
    print(f"{name} 已完成标准化转换")

    normalizer = Normalizer()
    X_normalized = normalizer.fit_transform(X)
    feature_sets['正则缩放_' + name] = X_normalized
    print(f"{name} 已完成正则缩放转换")

feature_set = feature_sets
feature_sets = {}

for name, X_scaled in feature_set.items():
    # 转换2: PCA降维
    pca = PCA(n_components=min(X_scaled.shape[1], 10))
    X_pca = pca.fit_transform(X_scaled)
    feature_sets['PCA_' + name] = X_pca
    print(f"{name} PCA解释方差比: {pca.explained_variance_ratio_}")
    print(f"{name} PCA累计方差占比: {np.sum(pca.explained_variance_ratio_):.4f}")

    # 转换3: FastICA
    ica = FastICA(n_components=min(X_scaled.shape[1], 10), random_state=42)
    X_ica = ica.fit_transform(X_scaled)
    feature_sets['ICA_' + name] = X_ica
    print(f"{name} 已完成ICA转换")
    # 转换4: t-SNE
    tsne = TSNE(n_components=2, random_state=42, init='random', learning_rate='auto')
    X_tsne = tsne.fit_transform(X_scaled)
    feature_sets['TSNE_' + name] = X_tsne
    print(f"{name} 已完成t-SNE转换")

# 可视化所有特征集
n = len(feature_sets)
cols = 3
rows = math.ceil(n / cols)
plt.figure(figsize=(5 * cols, 5 * rows))
for i, (fname, data) in enumerate(feature_sets.items()):
    plt.subplot(rows, cols, i + 1)
    if data.shape[1] > 1:
        plt.scatter(data[:, 0], data[:, 1], alpha=0.5)
        plt.title(f"{fname} (前两个维度)")
    else:
        plt.hist(data[:, 0], bins=20)
        plt.title(f"{fname}分布")
plt.tight_layout()
plt.show()

## 聚类评估函数
定义用于评估聚类结果的性能指标函数

In [ ]:
# 第四个代码单元格 - 评估函数
def evaluate_clustering(X, labels, name):
    """计算聚类性能指标"""
    try:
        silhouette = silhouette_score(X, labels)
    except:
        silhouette = -1

    try:
        db_score = davies_bouldin_score(X, labels)
    except:
        db_score = float('inf')

    try:
        ch_score = calinski_harabasz_score(X, labels)
    except:
        ch_score = -1

    return {
        'silhouette_score': silhouette,  # 越高越好
        'davies_bouldin_score': db_score,  # 越低越好
        'calinski_harabasz_score': ch_score,  # 越高越好
        'method': name
    }

## 聚类算法实现
实现三种聚类算法及其不同参数设置：
1. K-means聚类
2. 高斯混合模型(GMM)
3. 层次聚类(Hierarchical Clustering)

In [ ]:
def run_kmeans(X, n_clusters_range=[4], init_methods=['k-means++', 'random']):
    """运行K-means并尝试不同参数"""
    results = []

    for n_clusters in n_clusters_range:
        for init in init_methods:
            name = f"KMeans(n_clusters={n_clusters}, init={init})"
            try:
                model = KMeans(n_clusters=n_clusters, init=init, random_state=42)
                labels = model.fit_predict(X)

                # 评估结果
                result = evaluate_clustering(X, labels, name)
                result['labels'] = labels
                result['model'] = model
                result['n_clusters'] = n_clusters
                result['init'] = init

                if hasattr(X, 'columns'):  # 如果X是DataFrame
                    result['feature_names'] = X.columns.tolist()
                else:  # 如果X是ndarray（比如预处理后）
                    result['feature_names'] = [f"feature_{i}" for i in range(X.shape[1])]

                results.append(result)
                print(f"完成: {name}")
            except Exception as e:
                print(f"错误 {name}: {str(e)}")

    return results

In [ ]:
def run_gmm(X, n_components_range=[2, 3, 4, 5, 6, 7, 8], covariance_types=['full', 'tied', 'diag', 'spherical']):
    """运行高斯混合模型并尝试不同参数"""
    results = []

    for n_components in n_components_range:
        for cov_type in covariance_types:
            name = f"GMM(n_components={n_components}, covariance_type={cov_type})"
            try:
                model = GaussianMixture(n_components=n_components, covariance_type=cov_type, random_state=42)
                labels = model.fit_predict(X)

                # 评估结果
                result = evaluate_clustering(X, labels, name)
                result['labels'] = labels
                result['model'] = model
                result['n_components'] = n_components
                result['covariance_type'] = cov_type

                if hasattr(X, 'columns'):  # 如果是DataFrame
                    result['feature_names'] = X.columns.tolist()
                else:  # 如果是ndarray
                    result['feature_names'] = [f"feature_{i}" for i in range(X.shape[1])]

                results.append(result)
                print(f"完成: {name}")
            except Exception as e:
                print(f"错误 {name}: {str(e)}")

    return results

In [ ]:
def run_hierarchical(X, n_clusters_range=[4], linkage_methods=['ward', 'complete', 'average', 'single']):
    """运行层次聚类并尝试不同参数"""
    results = []

    for n_clusters in n_clusters_range:
        for linkage in linkage_methods:
            # ward 只能用于欧氏距离
            if linkage == 'ward':
                metric = 'euclidean'
            else:
                metric = 'euclidean'  # 也可以尝试其他距离，比如 'manhattan'

            name = f"HC(n_clusters={n_clusters}, linkage={linkage})"
            try:
                model = AgglomerativeClustering(
                    n_clusters=n_clusters,
                    linkage=linkage,
                    metric=metric  # metric 是新版 sklearn 的参数，老版叫 affinity
                )
                labels = model.fit_predict(X)

                # 评估结果
                result = evaluate_clustering(X, labels, name)
                result['labels'] = labels
                result['model'] = model
                result['n_clusters'] = n_clusters
                result['linkage'] = linkage

                # 补充 feature_names
                if hasattr(X, 'columns'):  # DataFrame
                    result['feature_names'] = X.columns.tolist()
                else:  # Numpy array
                    result['feature_names'] = [f"feature_{i}" for i in range(X.shape[1])]

                results.append(result)
                print(f"完成: {name}")
            except Exception as e:
                print(f"错误 {name}: {str(e)}")

    return results

## 运行聚类实验
对每种特征集运行三种聚类算法，并尝试不同的参数设置

In [ ]:
# 第八个代码单元格 - 运行所有实验
# 为了限制运行时间，可以减少参数组合
n_clusters_range = [4]  # 聚类数量范围
init_methods =['k-means++']  # K-means初始化方法
covariance_types = ['full', 'tied', 'diag', 'spherical']  # GMM协方差类型
linkage_methods = ['ward', 'complete', 'average', 'single'] # 层次聚类链接方法

# 4. 运行所有实验并收集结果
all_results = {}

for feature_name, X_transformed in feature_sets.items():
    print(f"\n处理特征集: {feature_name}")

    # 运行三种聚类算法
    kmeans_results = run_kmeans(X_transformed, n_clusters_range, init_methods)
    gmm_results = run_gmm(X_transformed, n_clusters_range, covariance_types)
    hc_results = run_hierarchical(X_transformed, n_clusters_range, linkage_methods)

    # 保存结果
    all_results[feature_name] = {
        'kmeans': kmeans_results,
        'gmm': gmm_results,
        'hierarchical': hc_results
    }

## 结果分析
找出每种特征集和每种聚类方法的最佳结果

In [ ]:
# 第九个代码单元格 - 找出最佳结果
# 5. 找出每种特征集和每种聚类方法的最佳结果
best_results = {}

for feature_name, methods in all_results.items():
    best_results[feature_name] = {}

    for method_name, results in methods.items():
        if method_name in ['kmeans', 'gmm', 'hierarchical']:
            # 按silhouette_score排序（越高越好）
            sorted_results = sorted(results, key=lambda x: x['silhouette_score'], reverse=True)
            if sorted_results:
                best_results[feature_name][method_name] = sorted_results[0]

In [ ]:
# 第十个代码单元格 - 创建结果表格
# 6. 创建结果表格
results_table = []

for feature_name, methods in best_results.items():
    for method_name, result in methods.items():
        row = {
            '特征集': feature_name,
            '聚类方法': method_name,
            '轮廓系数': result['silhouette_score'],
            'Davies-Bouldin': result['davies_bouldin_score'],
            'Calinski-Harabasz': result['calinski_harabasz_score']
        }

        # 添加模型特有的参数
        if method_name == 'kmeans':
            row['聚类数'] = result['n_clusters']
            row['初始化方法'] = result['init']
        elif method_name == 'gmm':
            row['聚类数/组件数'] = result['n_components']
            row['协方差类型'] = result['covariance_type']
        elif method_name == 'hierarchical':
            row['聚类数'] = result['n_clusters']
            row['链接方法'] = result['linkage']

        results_table.append(row)

# 7. 将结果转换为DataFrame并显示
results_df = pd.DataFrame(results_table)
print("\n聚类结果表:")
display(results_df)

## 可视化聚类结果
可视化展示每种特征集和聚类方法的最佳聚类结果

In [ ]:
# 第十一个代码单元格 - 可视化最佳结果
# 8. 可视化每种特征集和聚类方法的最佳结果
for feature_name, methods in best_results.items():
    for method_name, result in methods.items():
        # 如果特征维度大于2，使用PCA降至2维进行可视化
        if feature_sets[feature_name].shape[1] > 2:
            vis_pca = PCA(n_components=2)
            X_vis = vis_pca.fit_transform(feature_sets[feature_name])
        else:
            X_vis = feature_sets[feature_name]

        plt.figure(figsize=(8, 6))
        plt.scatter(X_vis[:, 0], X_vis[:, 1], c=result['labels'], cmap='viridis', alpha=0.8, s=50)
        plt.title(f'特征集: {feature_name}, 聚类方法: {method_name}\n轮廓系数: {result["silhouette_score"]:.4f}')
        plt.colorbar(label='聚类标签')
        plt.tight_layout()
        plt.show()

In [ ]:
# 第十二个代码单元格 - 汇总表格
# 9. 生成最佳结果的汇总表格
summary_df = pd.DataFrame(results_table)

# 按特征集和聚类方法分组，找出每种组合的最佳结果
best_by_feature = summary_df.sort_values('轮廓系数', ascending=False).groupby(['特征集', '聚类方法']).first().reset_index()

print("最佳聚类结果汇总表:")
display(best_by_feature)

# 创建热力图显示不同特征集和聚类方法的轮廓系数
pivot_table = best_by_feature.pivot(index='特征集', columns='聚类方法', values='轮廓系数')
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, cmap='YlGnBu', fmt='.4f')
plt.title('不同特征集和聚类方法的轮廓系数')
plt.tight_layout()
plt.show()

In [ ]:
import warnings
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd

warnings.filterwarnings('ignore')

# 真实标签编码，只需一次
y_true = LabelEncoder().fit_transform(df['Programme'])

# 结果表格
results_table = []

for feature_name, methods in best_results.items():
    for method_name, result in methods.items():
        # 记录聚类评估指标
        row = {
            '特征集': feature_name,
            '聚类方法': method_name,
            '轮廓系数': result['silhouette_score'],
            'Davies-Bouldin': result['davies_bouldin_score'],
            'Calinski-Harabasz': result['calinski_harabasz_score']
        }
        if method_name == 'kmeans':
            row['聚类数'] = result['n_clusters']
            row['初始化方法'] = result['init']
        elif method_name == 'gmm':
            row['组件数'] = result['n_components']
            row['协方差类型'] = result['covariance_type']
        elif method_name == 'hierarchical':
            row['聚类数'] = result['n_clusters']
            row['链接方法'] = result['linkage']
        results_table.append(row)

        # 可视化
        X_plot = feature_sets[feature_name]
        if X_plot.shape[1] > 2:
            X_vis = PCA(n_components=2).fit_transform(X_plot)
        else:
            X_vis = X_plot
        plt.figure(figsize=(8, 6))
        plt.scatter(X_vis[:, 0], X_vis[:, 1], c=result['labels'], cmap='viridis', alpha=0.8, s=50)
        plt.title(f'特征集: {feature_name}, 聚类方法: {method_name}\n轮廓系数: {result["silhouette_score"]:.4f}')
        plt.colorbar(label='聚类标签')
        plt.tight_layout()
        plt.show()

        # 聚类与真实标签对比
        labels = result['labels']
        ari = adjusted_rand_score(y_true, labels)
        nmi = normalized_mutual_info_score(y_true, labels)
        print(f"\n特征集: {feature_name}, 聚类方法: {method_name}")
        print(f"调整兰德指数(ARI): {ari:.4f}")
        print(f"归一化互信息(NMI): {nmi:.4f}")
        print("混淆矩阵:")
        print(confusion_matrix(y_true, labels))

# 结果表格展示
results_df = pd.DataFrame(results_table)
print("\n聚类结果表:")
print(results_df)

In [102]:
# 1. 找到轮廓系数最高的模型
best_model = None
best_score = -float('inf')
best_feature_name = None
best_method = None
test_df = pd.read_csv("./student_data.csv")

for feature_name, methods in all_results.items():
    for method_name, results in methods.items():
        for result in results:
            if result['silhouette_score'] > best_score:
                best_score = result['silhouette_score']
                best_model = result['model']
                best_feature_name = result['feature_names']
                best_method = method_name

print(f"最佳模型: 特征集={best_feature_name}, 方法={best_method}, 轮廓系数={best_score:.4f}")

# 2. 用最佳特征集的scaler和特征处理测试集
test_feature_sets = process_data(test_df, mode='test', preprocessors=preprocessors)

# ✅ 直接用完整的 feature 名字
X_test_scaled = test_feature_sets[best_feature_name]

# 3. 用最佳模型预测
test_labels = best_model.predict(X_test_scaled)

# 4. 评估
result = evaluate_clustering(X=X_test_scaled, labels=test_labels, name='best_model')
print(f"轮廓系数: {result['silhouette_score']:.4f}")

最佳模型: 特征集=['feature_0', 'feature_1'], 方法=kmeans, 轮廓系数=0.5568


ValueError: could not convert string to float: 'feature_0'